In [86]:
import urllib.request
import tarfile
import urllib
from Bio import Entrez
from xml.dom import minidom
import xml.etree.ElementTree as ET
import pandas as pd
import os
import glob
import shutil
import urllib.request as request
from contextlib import closing
import pickle

In [87]:
# 1. goal: summarization for articles that do not have abstract
# 2. mesh: sub-categories
# 3. pubmed pmc[sb] - restrict to articles that have full text in PMC
# 4. cluster: title only / fulltext only / abstract & title only ...  - sperate them 
# 5. ignore the case as https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1248604/pdf/bmjcred00049-0017.pdf 
# 6. two dataset: a. for publication types (following multitagger paper) 1987 - 2023 b. for sumarization (depends on pubmed central)
# 7. standard dataset: 
# 8. remind halil to create the github repo


In [88]:
import os

In [125]:
year = "2023"

In [111]:
current_directory = os.getcwd() + "/data_summarization_with_abstract/" + year

In [112]:
test_xml = os.listdir(current_directory)[0]

In [113]:
tree = ET.parse(os.path.join(os.getcwd() + "/data_summarization_with_abstract/" + year, test_xml))
root = tree.getroot()

In [114]:
# the year / date  <PubDate>
# journal title / issn
# page information (start / end) - to calculate how long the article is
# the <meshheading> list
# publication type list <PublicationType>
# doi - for special cases 

test_count = 1000000
count = 0
pmids = []
pmcids = []
Article_Title = []
Article_Abstract = []

PubDate = []
Journal_ISSN = []
Journal_Title = []
StartPage = []
EndPage = []
MeshHeading = []
PublicationType = []
doi = []
KeywordList = []


for child in root:
    if count < test_count:
        if child.tag == "PubmedArticle":
            pmid = ""
            title = ""
            pmcid = ""
            abstract = ""
            pub_date = ""
            issn = ""
            journal_title = ""
            start_page = ""
            end_page = ""
            mesh_heading = []
            publication_type = []
            single_doi = ""
            keyword_list = []
            
            count = count + 1
            for child_level_2 in child:
                if "MedlineCitation" in child_level_2.tag:
                    for child_level_3 in child_level_2:
                        if ("PMID" in child_level_3.tag):
                            pmid = child_level_3.text
                                    
                        if "MeshHeadingList" in child_level_3.tag:
                            for child_level_4 in child_level_3:
                                if "MeshHeading" in child_level_4.tag:
                                    for child_level_5 in child_level_4:
                                        if "DescriptorName" in child_level_5.tag:
                                            mesh_heading.append(child_level_5.text)
                                            
                        if "KeywordList" in child_level_3.tag:
                            for child_level_4 in child_level_3:
                                if "Keyword" in child_level_4.tag:
                                    keyword_list.append(child_level_4.text)
                                            
                        if ("Article" in child_level_3.tag):
                            for child_level_4 in child_level_3:
                                if "ArticleTitle" in child_level_4.tag:
                                    title = child_level_4.text
                                if "Abstract" in child_level_4.tag:
                                    for child_level_5 in child_level_4:
                                        if "AbstractText" in child_level_5.tag:
                                            abstract = child_level_5.text
                                if ("Journal" in child_level_4.tag):
                                    for child_level_5 in child_level_4:
                                        if "ISSN" in child_level_5.tag: 
                                            issn = child_level_5.text
                                        if "Title" in child_level_5.tag:
                                            journal_title = child_level_5.text
                                        if "JournalIssue" in child_level_5.tag:
                                            for child_level_6 in child_level_5:
                                                if "PubDate" in child_level_6.tag:
                                                    for child_level_7 in child_level_6:
                                                        if "Year" in child_level_7.tag:
                                                            pub_date = pub_date + child_level_7.text
                                                        if "Month" in child_level_7.tag:
                                                            pub_date = pub_date + ", " + child_level_7.text
                                if "Pagination" in child_level_4.tag:
                                    for child_level_5 in child_level_4:
                                        if "StartPage" in child_level_5.tag:
                                            start_page = child_level_5.text
                                        if "EndPage" in child_level_5.tag:
                                            end_page = child_level_5.text
                                if "PublicationTypeList" in child_level_4.tag:
                                    for child_level_5 in child_level_4:
                                        if "PublicationType" in child_level_5.tag:
                                            publication_type.append(child_level_5.text)
                                            
                elif "PubmedData" in child_level_2.tag:
                    for child_level_3 in child_level_2:
                        if ("ArticleIdList" in child_level_3.tag):
                            for child_level_4 in child_level_3:
                                if child_level_4.attrib["IdType"] == "pmc":
                                    pmcid = child_level_4.text
                                if child_level_4.attrib["IdType"] == "doi":
                                    single_doi = child_level_4.text
            pmids.append(pmid)
            Article_Title.append(title)
            pmcids.append(pmcid)
            Article_Abstract.append(abstract)
            
            PubDate.append(pub_date)
            Journal_ISSN.append(issn)
            Journal_Title.append(journal_title)
            StartPage.append(start_page)
            EndPage.append(end_page)
            MeshHeading.append(mesh_heading)
            PublicationType.append(publication_type)
            doi.append(single_doi)
            KeywordList.append(keyword_list)
            

In [126]:
df.to_csv(year + "_metadata.csv")

In [115]:
df = pd.DataFrame(list(zip(pmids, pmcids, PubDate, Journal_ISSN, Journal_Title, EndPage, MeshHeading, PublicationType, StartPage, Article_Title, Article_Abstract, doi, KeywordList)), columns =["pmids", "pmcids", "PubDate", "Journal_ISSN", "Journal_Title", "EndPage", "MeshHeading", "PublicationType", "StartPage", "Article_Title", "Article_Abstract", "doi", "KeywordList"])

In [117]:
df

,pmids,pmcids,PubDate,Journal_ISSN,Journal_Title,EndPage,MeshHeading,PublicationType,StartPage,Article_Title,Article_Abstract,doi,KeywordList
0,37811508,PMC10551689,"2023, Oct",2005-4408,Clinics in orthopedic surgery,733,"[Female, Humans, Adolescent, Finite Element An...","[Case Reports, Journal Article]",725,Femoral Bowing Increases Early Postoperative S...,Surgeons should assess femoral bowing preopera...,10.4055/cios22392,"[Bowing, Femur, Finite element analysis, Stem,..."
1,37808910,PMC10552297,2023,2235-2988,Frontiers in cellular and infection microbiology,,"[Female, Humans, Aged, Aged, 80 and over, Myco...","[Case Reports, Research Support, Non-U.S. Gov't]",1268668,Cure of,We report a case of,10.3389/fcimb.2023.1268668,"[Mycobacterium morphology, keratitis, laborato..."
2,37808435,PMC10559155,2023,1937-8688,The Pan African medical journal,,"[Male, Humans, Adult, Middle Aged, Aged, Aged,...",[Case Reports],144,Contrast media-induced nephropathy in Tunisia:...,the most effective approach for CMIN is preven...,10.11604/pamj.2023.45.144.30749,"[Acute renal failure, iodinated contrast media..."
3,37805489,PMC10560414,"2023, Oct",1752-1947,Journal of medical case reports,,"[Humans, Female, Aged, Ureter, Ureteral Diseas...","[Case Reports, Journal Article]",443,Localized amyloidosis of the ureter: a case re...,We herein report a rare case of localized uret...,10.1186/s13256-023-04138-y,"[Amyloidosis, Localized amyloidosis, Ureteral ..."
4,37805478,PMC10560420,"2023, Oct",1749-8090,Journal of cardiothoracic surgery,,"[Humans, Female, Aged, 80 and over, Percutaneo...","[Case Reports, Journal Article]",275,Successful conservative treatment for left ven...,Left ventricular free wall rupture (LVFWR) is ...,10.1186/s13019-023-02397-w,"[Acute myocardial infarction (AMI), Cardiac ta..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6003,33713550,PMC10029896,"2023, Mar",2053-1095,Veterinary medicine and science,716,"[Humans, Animals, Horses, Aortic Valve, Quadri...",[Case Reports],712,Transthoracic echocardiography and its limitat...,Aortic valve malformation is a common congenit...,10.1002/vms3.472,"[aortic malformation, congenital abnormality, ..."
6004,33709828,PMC10076152,"2023, Feb",1724-6075,Urologia,197,"[Humans, Male, Middle Aged, Adjuvants, Immunol...","[Review, Case Reports, Journal Article]",195,Disease management in a patient diagnosed with...,This group of patients' recommendation is to d...,10.1177/03915603211001670,"[Bladder cancer, COVID-19, coronavirus, intrav..."
6005,33394953,PMC9750092,"2023, Jan",1937-1578,Retinal cases & brief reports,22,"[Male, Humans, Aged, Melanoma, Skin Neoplasms,...","[Case Reports, Journal Article]",18,VOGT-KOYANAGI-HARADA-LIKE UVEITIS FOLLOWED BY ...,Patients with metastatic cutaneous melanoma ca...,10.1097/ICB.0000000000001113,[]
6006,33323898,PMC9750093,"2023, Jan",1937-1578,Retinal cases & brief reports,43,"[Female, Humans, Middle Aged, Scleral Buckling...","[Case Reports, Journal Article]",41,INTRACTABLE PAIN AND DIPLOPIA AFTER SCLERAL BU...,Spontaneous dehiscence of scleral belt loops m...,10.1097/ICB.0000000000001104,[]


In [118]:
df_filtered = df[df.pmcids != ""]



In [119]:
df_filtered



,pmids,pmcids,PubDate,Journal_ISSN,Journal_Title,EndPage,MeshHeading,PublicationType,StartPage,Article_Title,Article_Abstract,doi,KeywordList
0,37811508,PMC10551689,"2023, Oct",2005-4408,Clinics in orthopedic surgery,733,"[Female, Humans, Adolescent, Finite Element An...","[Case Reports, Journal Article]",725,Femoral Bowing Increases Early Postoperative S...,Surgeons should assess femoral bowing preopera...,10.4055/cios22392,"[Bowing, Femur, Finite element analysis, Stem,..."
1,37808910,PMC10552297,2023,2235-2988,Frontiers in cellular and infection microbiology,,"[Female, Humans, Aged, Aged, 80 and over, Myco...","[Case Reports, Research Support, Non-U.S. Gov't]",1268668,Cure of,We report a case of,10.3389/fcimb.2023.1268668,"[Mycobacterium morphology, keratitis, laborato..."
2,37808435,PMC10559155,2023,1937-8688,The Pan African medical journal,,"[Male, Humans, Adult, Middle Aged, Aged, Aged,...",[Case Reports],144,Contrast media-induced nephropathy in Tunisia:...,the most effective approach for CMIN is preven...,10.11604/pamj.2023.45.144.30749,"[Acute renal failure, iodinated contrast media..."
3,37805489,PMC10560414,"2023, Oct",1752-1947,Journal of medical case reports,,"[Humans, Female, Aged, Ureter, Ureteral Diseas...","[Case Reports, Journal Article]",443,Localized amyloidosis of the ureter: a case re...,We herein report a rare case of localized uret...,10.1186/s13256-023-04138-y,"[Amyloidosis, Localized amyloidosis, Ureteral ..."
4,37805478,PMC10560420,"2023, Oct",1749-8090,Journal of cardiothoracic surgery,,"[Humans, Female, Aged, 80 and over, Percutaneo...","[Case Reports, Journal Article]",275,Successful conservative treatment for left ven...,Left ventricular free wall rupture (LVFWR) is ...,10.1186/s13019-023-02397-w,"[Acute myocardial infarction (AMI), Cardiac ta..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6003,33713550,PMC10029896,"2023, Mar",2053-1095,Veterinary medicine and science,716,"[Humans, Animals, Horses, Aortic Valve, Quadri...",[Case Reports],712,Transthoracic echocardiography and its limitat...,Aortic valve malformation is a common congenit...,10.1002/vms3.472,"[aortic malformation, congenital abnormality, ..."
6004,33709828,PMC10076152,"2023, Feb",1724-6075,Urologia,197,"[Humans, Male, Middle Aged, Adjuvants, Immunol...","[Review, Case Reports, Journal Article]",195,Disease management in a patient diagnosed with...,This group of patients' recommendation is to d...,10.1177/03915603211001670,"[Bladder cancer, COVID-19, coronavirus, intrav..."
6005,33394953,PMC9750092,"2023, Jan",1937-1578,Retinal cases & brief reports,22,"[Male, Humans, Aged, Melanoma, Skin Neoplasms,...","[Case Reports, Journal Article]",18,VOGT-KOYANAGI-HARADA-LIKE UVEITIS FOLLOWED BY ...,Patients with metastatic cutaneous melanoma ca...,10.1097/ICB.0000000000001113,[]
6006,33323898,PMC9750093,"2023, Jan",1937-1578,Retinal cases & brief reports,43,"[Female, Humans, Middle Aged, Scleral Buckling...","[Case Reports, Journal Article]",41,INTRACTABLE PAIN AND DIPLOPIA AFTER SCLERAL BU...,Spontaneous dehiscence of scleral belt loops m...,10.1097/ICB.0000000000001104,[]


In [120]:
pmcids = df_filtered.pmcids.to_list()

In [ ]:
tarfiles_list = []
have_link = []
for PMCID in pmcids:
    if PMCID != "":
        # call PMC API here
        link = "https://www.ncbi.nlm.nih.gov/pmc/utils/oa/oa.fcgi?id=" + PMCID
        #35640925
        # open the link
        request = urllib.request.Request(link)
        result = urllib.request.urlopen(request)
        tree = ET.parse(result)
        doc = tree.getroot()
        for child in doc:
            if child.tag == "records":
                article_download_link = ""
                for child_level_2 in child:
                    if child_level_2.tag == "record":
                        for child_level_3 in child_level_2:
                            if ("link" in child_level_3.tag):
                                link_format = child_level_3.attrib["format"]
                                if (article_download_link=="") and (link_format == "tgz"):
                                    article_download_link = child_level_3.attrib["href"]
                                    
                # print out the article tar folder download link
                
                ftpstream = urllib.request.urlopen(article_download_link)
                thetarfile = tarfile.open(fileobj=ftpstream, mode="r|gz")
                
                # put the path of output folder here
                thetarfile.extractall(path= current_directory + "/NXML_second/")